In [1]:
import pandas as pd
import numpy as np

In [2]:
time_series = pd.read_csv('/home/ananth/Downloads/time_series_with_causes_zscore_full.csv')

In [7]:
time_series.columns.values

array(['Unnamed: 0', 'index', 'country', 'admin_code', 'admin_name',
       'centx', 'centy', 'year_month', 'year', 'month', 'fews_ipc',
       'fews_ha', 'fews_proj_near', 'fews_proj_near_ha', 'fews_proj_med',
       'fews_proj_med_ha', 'ndvi_mean', 'ndvi_anom', 'rain_mean',
       'rain_anom', 'et_mean', 'et_anom', 'acled_count',
       'acled_fatalities', 'p_staple_food', 'area', 'cropland_pct', 'pop',
       'ruggedness_mean', 'pasture_pct', 'change_fews', 'land seizures_0',
       'land seizures_1', 'land seizures_2', 'slashed export_0',
       'slashed export_1', 'slashed export_2', 'price rise_0',
       'price rise_1', 'price rise_2', 'mass hunger_0', 'mass hunger_1',
       'mass hunger_2', 'cyclone_0', 'cyclone_1', 'cyclone_2',
       'failed crops_0', 'failed crops_1', 'failed crops_2',
       'disruption to farming_0', 'disruption to farming_1',
       'disruption to farming_2', 'massive starvation_0',
       'massive starvation_1', 'massive starvation_2',
       'abnormall

In [10]:
t_variant_traditional_factors = ['ndvi_mean', 'ndvi_anom', 'rain_mean', 'rain_anom', 'et_mean', 'et_anom', 
                                    'acled_count', 'acled_fatalities', 'p_staple_food']
t_invariant_traditional_factors = ['area', 'cropland_pct', 'pop', 'ruggedness_mean', 'pasture_pct']
news_factors = [name[:-2] for name in time_series.columns.values if '_0' in name]

In [12]:
len(news_factors)

167

In [13]:
(1-3)%12

10

In [ ]:
def get_lagged(x, f, t):
    admin_code = x['admin_code']
    year = x['year']
    month = x['month']
    l_month = ((month-1-t)%12)+1
    l_year = year
    if month-t<=0:
        l_year -= 1
    ts=time_series[time_series['admin_code']==admin_code]
    lagged_year_month = '{}_{}'.format(l_year, l_month)
    if lagged_year_month in ts['year_month'].values:
        ts = ts[ts['year_month']==lagged_year_month]
        return ts[f].values[0]
    else:
        return x[f]
    

In [44]:
def add_time_lagged(features, start=3, end=9, diff=1):    
    for f in features:
        for t in range(start,end,diff):
            time_series['{}_{}'.format(f,t)] = time_series.apply(lambda x: get_lagged(x, f, t), axis=1)
    return time_series

In [ ]:
add_time_lagged(t_variant_traditional_factors)

In [ ]:
add_time_lagged(news_factors)

In [ ]:
add_time_lagged(['fews_ipc'], end=21, diff=3)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

test_splits = [
    ((2010,7), (2011, 7)), 
    ((2011,7), (2012, 7)),
    ((2012,7), (2013, 7)), 
    ((2013,7), (2014, 7)), 
    ((2014,7), (2015, 7)), 
    ((2015,7), (2016, 7)), 
    ((2016,7), (2017, 7)), 
    ((2017,7), (2018, 7)),
    ((2018,7), (2019, 7)), 
    ((2019,2), (2020, 2)),
]
train_splits = [
    ((2009,7), (2010,4)),
    ((2009,7), (2011,1)),
    ((2009,7), (2011,10)),
    ((2009,7), (2012,7)),
    ((2009,7), (2013,7)),
    ((2009,7), (2014,1)),
    ((2009,7), (2015,1)),
    ((2009,7), (2015,10)),
    ((2009,7), (2016,10)),
    ((2009,7), (2017,2))]
dev_splits = [
    ((2010,4), (2010, 7)),
    ((2011,1), (2011, 7)),
    ((2011,10), (2012, 7)),
    ((2012,7), (2013, 7)),
    ((2013,4), (2014, 7)),
    ((2014,1), (2015, 7)),
    ((2015,1), (2016, 7)),
    ((2015,10), (2017, 7)),
    ((2016,10), (2018, 7)),
    ((2017,2), (2019, 2)),
]
regr = RandomForestRegressor(max_features='auto', n_estimators=100, 
                             min_samples_split=0.5, min_impurity_decrease=0.001, random_state=0)

features = {
    'traditional': time_series[
        ['{}_{}'.format('fews_ipc', t) for t in range(3,21,3)] + 
        ['{}_{}'.format(f, t) for f, t in zip(t_variant_traditional_factors, range(3,9))] + 
        t_invariant_traditional_factors
    ], 
    'news': time_series[
        ['{}_{}'.format('fews_ipc', t) for t in range(3,21,3)] +
        ['{}_{}'.format(f, t) for f, t in zip(news_factors, range(3,9))]
    ], 
    'traditional+news': time_series[
        ['{}_{}'.format('fews_ipc', t) for t in range(3,21,3)] +
        ['{}_{}'.format(f, t) for f, t in zip(t_variant_traditional_factors, range(3,9))] + 
        t_invariant_traditional_factors +
        ['{}_{}'.format(f, t) for f, t in zip(news_factors, range(3,9))]
    ], 
}

labels_df = time_series['fews_ipc']

def get_time_split(df, start, end):
    return df[df['year'] >= start[0] & df['month'] >= start[1] & df['year'] <= end[0] & df['month'] <= end[1]]

for train, dev, test in zip(train_splits, dev_splits, test_splits):
    for f, D in features.items():
        X = get_time_split(D, train[0], train[1])
        y = get_time_split(labels_df, test[0], test[1])
        regr.fit(X, y)
        X_test = get_time_split(D, test[0], test[1])
        preds = regr.predict(X_test)
        labels = get_time_split(labels_df, test[0], test[1])
        rmse = mean_squared_error(labels, preds, squared=True)
        print (test, f, rmse)

[-2.50699856]
